In [1]:
# imports
import numpy as np
from PIL import Image

In [2]:
# variables
width = 300
height = 300
viewportSize = 1
projectionPlane = 1

#Scene objects

camera = np.array([0,3,-7])

spheres = [{"center": np.array([0,1,3]), "radius": 1, "color": (255,0,0),"S":500, "refl": 0.0, "refr": 0},#red
          {"center": np.array([-2,1,4]), "radius": 1, "color": (0,255,0),"S":10 , "refl": 0,  "refr": 0}, #green
          {"center": np.array([2,1,4]), "radius": 1, "color": (0,0,255),"S":500,  "refl": 0.1,  "refr": 0}, #blue
           {"center": np.array([-3,4,15]), "radius": 4, "color": (200,200,200),"S":500,  "refl": 1, "refr": 1.5},#mirror
           {"center": np.array([-3,1,1]), "radius": 1, "color": (200, 100, 215),"S":500,  "refl": 0.04,"refr": 0},#purple
           {"center": np.array([3,1,3]), "radius": 1, "color": (240, 240, 0),"S":300,  "refl": 0.6 , "refr": 0},#yellow
           {"center": np.array([-1,2,7]), "radius": 2, "color": (66, 236, 215),"S":200,  "refl":  0.3,  "refr": 0},#cyan
          {"center": np.array([0,-1000,0]), "radius": 1000, "color": (200,200,215),"S":0,  "refl": 0 , "refr": 0} #grey floor
          ]
lights = [
    {"type": "ambient","intensity": 0.3},
    {"type": "point","intensity": 0.4, "position": np.array([4,2,-1])},
    {"type": "directional","intensity": 0.3, "direction": np.array([0,1,0])}
]

#,{"center": np.array([0,-5001,0]), "radius": 5000, "color": (255,255,0),"S":1000}

def lengthSquared(corrd):
    
    return corrd[0]*corrd[0] + corrd[1]*corrd[1] + corrd[2]*corrd[2]

In [3]:
def canvasViewport(x,y):
    return np.array([
        x * viewportSize / width,
        y * viewportSize / height,
        projectionPlane
    ])

In [4]:
def intersectRays(origin, direction, sphere):
    
    center = sphere["center"]
    r = sphere["radius"]
    oc = origin - center
    
    a = np.dot(direction,direction)
    b = 2 * np.dot(oc,direction)
    c = np.dot(oc,oc) - r**2
    
    disc = b**2 - 4*(a*c)
    if (disc < 0):
        return float("inf"), float("inf")
    else:
        t1 = (-b - np.sqrt(disc)) / (2*a)
        t2 = (-b + np.sqrt(disc)) / (2*a)
        return t1,t2
    
    

In [5]:
def closestIntersection(O,D,t_min):
    closest_t = float("inf") 
    closestSphere = None
        
    #check for ray intersections with spheres
    for sphere in spheres:
        t1,t2 = intersectRays(O, D, sphere)
            
        if t1> t_min and t1 < closest_t:
            closest_t = t1
            closestSphere = sphere
            pixelColor = closestSphere["color"]
                
        if t2 > t_min and t2 < closest_t:
            closest_t = t2
            closestSphere = sphere
            pixelColor = closestSphere["color"]
            
    return closestSphere, closest_t
        

In [6]:
def computeLighting (P,N,V,S):
    intensity = 0.0
    
    for light in lights:
        if light["type"]=="ambient":
            intensity = intensity + light["intensity"]
        else:
            if light["type"] == "point":
                L = light["position"] - P
            else:
                L = light["direction"]
                
            L = L/np.linalg.norm(L)
            
            shadowSphere, shadow_t = closestIntersection(P,L,0.001)
            if shadowSphere != None:
                return min(intensity,1)
            
            n_dot_l = np.dot(N,L)
            if n_dot_l > 0:
                intensity = intensity + light["intensity"] * n_dot_l / (np.linalg.norm(L) * np.linalg.norm(N))
                
            if S != -1:
                R = 2 * N * np.dot(N,L) - L
                r_dot_v = np.dot(R,V)
                if r_dot_v > 0:
                    intensity = intensity + light["intensity"] * (r_dot_v/(np.linalg.norm(R)*np.linalg.norm(V))) ** S
                    
            
            
    return min(intensity,1)

In [7]:
def reflectRay(R,N):
    return 2 * N * np.dot(N,R) - R

def refractRay(R,N,etai_over_etat):
    
    cos_theta = min(np.dot(-R,N),1.0)
    
    r_out_perp = etai_over_etat * (R + cos_theta*N)
    
    r_out_parallel = -1 * np.sqrt(np.abs(1 - (lengthSquared(r_out_perp)))) * N
    
    return r_out_perp + r_out_parallel

In [8]:
def traceRay(O, D, t_min, t_max, depth):
    closestS, closestT = closestIntersection(O, D, t_min)

    if closestS == None: 
        return  (158, 224, 255)
    P = O + closestT * D
    N = P - closestS["center"]
    N = N/np.linalg.norm(N)
    pixelColor = closestS["color"]
    localColor = tuple(map(lambda v: min(round(v * computeLighting(P,N,-D,closestS["S"])),255), pixelColor))
    
    refr = closestS["refr"]
    

    refl = closestS["refl"]
    if depth<= 0 or refl <= 0:
        return localColor

    R = reflectRay(-D, N)
    
    if refr > 0:
        R = refractRay(D, N, refr)
        reflectedColor = traceRay(P, R, 0.001, t_max, depth - 1)
        return tuple(map(lambda v: min(round(v * refl ),255),reflectedColor))
        
    reflectedColor = traceRay(P, R, 0.001, t_max, depth - 1) 
    
    

    temp1 = tuple(map(lambda v: v * (1 - refl), localColor))
    temp2 = tuple(map(lambda v: v * refl, reflectedColor))
    return tuple(map(lambda v1,v2: min(round(v1 + v2 ),255), temp1, temp2))

In [ ]:
image = Image.new("RGB", (width, height), "white")
pixels = image.load()

#create canvas
for x in range(width):
    print( x/width * 100, "%")
    for y in range(height):
        viewX = (x - width/2)
        viewY = -(y - height/2)
        direction = canvasViewport(viewX,viewY)
        direction = direction /np.linalg.norm(direction)
        closest_t = float("inf")
        #pixelColor = (255,255,255) #background color if no intersection
        intersection = False
        
        #check for ray intersections with spheres

        pixelColor = traceRay(camera, direction, 1, 1000, 3)
        pixels[x,y] = pixelColor
        
        """
        for sphere in spheres:
            t1,t2 = intersectRays(camera, direction, sphere)
            
            if t1> 1 and t1 < closest_t:
                closest_t = t1
                closestSphere = sphere
                pixelColor = closestSphere["color"]
                intersection = True
                
            if t2 > 1 and t2 < closest_t:
                closest_t = t2
                closestSphere = sphere
                pixelColor = closestSphere["color"]
                intersection = True
                
            if intersection == True:
                P = camera + (closest_t * direction)
                N = P - closestSphere["center"]
                N = N / np.linalg.norm(N)
                pixels[x,y] = tuple(map(lambda v: min(round(v * computeLighting(P,N,-direction,closestSphere["S"])),255), pixelColor))
            else:
                pixels[x,y] = pixelColor
                """
        
        

0.0 %
0.33333333333333337 %
0.6666666666666667 %
1.0 %
1.3333333333333335 %
1.6666666666666667 %
2.0 %
2.3333333333333335 %
2.666666666666667 %
3.0 %
3.3333333333333335 %
3.6666666666666665 %
4.0 %
4.333333333333334 %
4.666666666666667 %
5.0 %
5.333333333333334 %
5.666666666666666 %
6.0 %
6.333333333333334 %
6.666666666666667 %
7.000000000000001 %
7.333333333333333 %
7.666666666666666 %
8.0 %
8.333333333333332 %
8.666666666666668 %
9.0 %
9.333333333333334 %
9.666666666666666 %
10.0 %
10.333333333333334 %
10.666666666666668 %
11.0 %
11.333333333333332 %
11.666666666666666 %
12.0 %
12.333333333333334 %
12.666666666666668 %
13.0 %
13.333333333333334 %
13.666666666666666 %
14.000000000000002 %
14.333333333333334 %
14.666666666666666 %
15.0 %
15.333333333333332 %
15.666666666666668 %
16.0 %
16.333333333333332 %
16.666666666666664 %
17.0 %
17.333333333333336 %
17.666666666666668 %
18.0 %
18.333333333333332 %
18.666666666666668 %
19.0 %
19.333333333333332 %
19.666666666666664 %
20.0 %
20.3333

In [ ]:
image.show()